In [ ]:
#STEP 1: IMPORTING RELEVANT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#STEP 2: IMPORTING DATA
from google.colab import files
uploaded = files.upload()

In [ ]:
#STEP 3: PREPARE DATA
df = pd.read_csv('all_stocks_5yr.csv')
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)


In [ ]:
#STEP 4: FILTERING OUT ONE TICKER, IN THIS CASE APPLE
ticker = 'AAPL'
df = df[df['Name'] == ticker].copy()

In [ ]:
#STEP 5: COMPUTING DAILY RETURNS
df['return'] = df['close'].pct_change()

In [ ]:
#UTILITY FUNCTION 1 : COMPUTING ANNUALIZED SHARPE RATIOS
def sharpe_ratio(returns, rf=0.0):
    """Annualized Sharpe ratio"""
    return np.sqrt(252) * (returns.mean() - rf) / returns.std()

In [ ]:
#UTILITY FUNCTION 2 : CALCULATE LARGEST OBSERVED DROP
def max_drawdown(cum_returns):
    """Maximum drawdown"""
    roll_max = cum_returns.cummax()
    drawdown = cum_returns / roll_max - 1
    return drawdown.min()

In [ ]:
#MOMENTUM STRATEGY
def backtest_momentum(window):
    """Backtest momentum strategy with given lookback window"""
    df['Momentum_Signal'] = np.where(df['close'].pct_change(window) > 0, 1, -1)
    df['Momentum_Strategy'] = df['Momentum_Signal'].shift(1) * df['return']
    cum_ret = (1 + df['Momentum_Strategy']).cumprod()
    return cum_ret, sharpe_ratio(df['Momentum_Strategy']), max_drawdown(cum_ret)

In [ ]:
#MEAN STRATEGY
def backtest_mean_reversion(ma_window, z_thresh=1.0):
    """Backtest mean-reversion strategy using z-scores"""
    rolling_mean = df['close'].rolling(ma_window).mean()
    rolling_std = df['close'].rolling(ma_window).std()
    z_score = (df['close'] - rolling_mean) / rolling_std
    df['MeanRev_Signal'] = np.where(z_score < -z_thresh, 1,
                           np.where(z_score > z_thresh, -1, 0))
    df['MeanRev_Strategy'] = df['MeanRev_Signal'].shift(1) * df['return']
    cum_ret = (1 + df['MeanRev_Strategy']).cumprod()
    return cum_ret, sharpe_ratio(df['MeanRev_Strategy']), max_drawdown(cum_ret)

In [ ]:
#OPTIMAL PARAMETERS SEARCH
momentum_windows = [5, 10, 20, 30, 60]
momentum_results = {}
for w in momentum_windows:
    _, s, _ = backtest_momentum(w)
    momentum_results[w] = s
best_mom_window = max(momentum_results, key=momentum_results.get)
print(f"Best Momentum Window: {best_mom_window} days (Sharpe={momentum_results[best_mom_window]:.2f})")

In [ ]:
#FINAL EVALUATION OF STRATEGIES
meanrev_windows = [10, 20, 30, 60]
meanrev_results = {}
for w in meanrev_windows:
    _, s, _ = backtest_mean_reversion(w)
    meanrev_results[w] = s
best_meanrev_window = max(meanrev_results, key=meanrev_results.get)
print(f"Best Mean-Reversion Window: {best_meanrev_window} days (Sharpe={meanrev_results[best_meanrev_window]:.2f})")

In [ ]:
mom_cum, mom_sharpe, mom_dd = backtest_momentum(best_mom_window)
meanrev_cum, meanrev_sharpe, meanrev_dd = backtest_mean_reversion(best_meanrev_window)
df['Benchmark_Cum'] = (1 + df['return']).cumprod()

In [ ]:
#VISUALISING
plt.figure(figsize=(12,6))
plt.plot(df['Benchmark_Cum'], label='S&P 500 Benchmark', linewidth=2)
plt.plot(mom_cum, label=f'Momentum (Sharpe={mom_sharpe:.2f}, DD={mom_dd:.2%})', linestyle='--')
plt.plot(meanrev_cum, label=f'Mean Reversion (Sharpe={meanrev_sharpe:.2f}, DD={meanrev_dd:.2%})', linestyle=':')
plt.title(f'{ticker} — Strategy Performance')
plt.xlabel('Date')
plt.ylabel('Cumulative Returns')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
#PRINTING
print("\nPerformance Summary")
print(f"Momentum: Sharpe={mom_sharpe:.2f}, Max Drawdown={mom_dd:.2%}")
print(f"Mean Reversion: Sharpe={meanrev_sharpe:.2f}, Max Drawdown={meanrev_dd:.2%}")